In [2]:
import torch
import numpy as np

In [31]:
target = np.array([[[1, 0, 1], [1, 1, 1], [0, 0, 1]]])
target = torch.from_numpy(target).type(torch.float32)

output = torch.tensor([[[[0, 1, 0], [0, 0, 1], [1, 1, 1]], [[1, 0, 1], [1, 1, 0], [0, 0, 0]]]], dtype = torch.float32)

print(target.size(), output.size())

torch.Size([1, 3, 3]) torch.Size([1, 2, 3, 3])


# 1. Pixel-level Classification

## BCEwithLogitLoss
- sigmoid + BCE

In [33]:
criterion = torch.nn.BCEWithLogitsLoss()
criterion(output[:, 1, :, :], target)  

tensor(0.5243)

In [37]:
sigma = torch.sigmoid(output[:, 1, :, :])
y = target

print(sigma)
print(y)

l_n = -(y*torch.log(sigma) + (1-y)*torch.log(1-sigma))
print(l_n)

l = torch.mean(l_n)
print(l)

tensor([[[0.7311, 0.5000, 0.7311],
         [0.7311, 0.7311, 0.5000],
         [0.5000, 0.5000, 0.5000]]])
tensor([[[1., 0., 1.],
         [1., 1., 1.],
         [0., 0., 1.]]])
tensor([[[0.3133, 0.6931, 0.3133],
         [0.3133, 0.3133, 0.6931],
         [0.6931, 0.6931, 0.6931]]])
tensor(0.5243)


In [4]:
torch.sigmoid(torch.tensor([0.5]))

tensor([0.6225])

In [6]:
fn_sigmoid = lambda x: 1/(1+np.exp(-x))
fn_sigmoid(0.5)

0.6224593312018546

## CrossEntropyLoss

In [23]:
criterion = torch.nn.CrossEntropyLoss()
criterion(output, target.type(torch.LongTensor))

tensor(0.5355)

In [38]:
target == 0

tensor([[[False,  True, False],
         [False, False, False],
         [ True,  True, False]]])

In [39]:
softmax = torch.softmax(output, dim = 1)
y = target.type(torch.LongTensor) 

print(softmax)
print(y)

l_n  = -(torch.log(softmax[:, 0, :, :])*(y == 0) + torch.log(softmax[:, 1, :, :])*(y == 1))
print(l_n)

l = torch.mean(l_n)
print(l)

tensor([[[[0.2689, 0.7311, 0.2689],
          [0.2689, 0.2689, 0.7311],
          [0.7311, 0.7311, 0.7311]],

         [[0.7311, 0.2689, 0.7311],
          [0.7311, 0.7311, 0.2689],
          [0.2689, 0.2689, 0.2689]]]])
tensor([[[1, 0, 1],
         [1, 1, 1],
         [0, 0, 1]]])
tensor([[[0.3133, 0.3133, 0.3133],
         [0.3133, 0.3133, 1.3133],
         [0.3133, 0.3133, 1.3133]]])
tensor(0.5355)


In [41]:
log_softmax = torch.log_softmax(output, dim = 1)
y = target.type(torch.LongTensor) 

print(log_softmax)
print(y)

l_n  = -(log_softmax[:, 0, :, :]*(y == 0) + log_softmax[:, 1, :, :]*(y == 1))
print(l_n)

l = torch.mean(l_n)
print(l)

tensor([[[[-1.3133, -0.3133, -1.3133],
          [-1.3133, -1.3133, -0.3133],
          [-0.3133, -0.3133, -0.3133]],

         [[-0.3133, -1.3133, -0.3133],
          [-0.3133, -0.3133, -1.3133],
          [-1.3133, -1.3133, -1.3133]]]])
tensor([[[1, 0, 1],
         [1, 1, 1],
         [0, 0, 1]]])
tensor([[[0.3133, 0.3133, 0.3133],
         [0.3133, 0.3133, 1.3133],
         [0.3133, 0.3133, 1.3133]]])
tensor(0.5355)


In [98]:
# for N classes

log_softmax = torch.log_softmax(output, dim = 1)
y = target.type(torch.LongTensor) 

y = y.unsqueeze(1)
n_cls = output.size(1)
shape = np.array(y.shape)
shape[1] = n_cls
one_hot = torch.zeros(tuple(shape))
one_hot = one_hot.scatter_(1, y, 1)

print(y)
print(one_hot)

log_softmax = torch.log_softmax(output, dim = 1)
l_n =-torch.sum(log_softmax*one_hot, dim = 1)
print(l_n)

l = torch.mean(l_n)
print(l)


tensor([[[[1, 0, 1],
          [1, 1, 1],
          [0, 0, 1]]]])
tensor([[[[0., 1., 0.],
          [0., 0., 0.],
          [1., 1., 0.]],

         [[1., 0., 1.],
          [1., 1., 1.],
          [0., 0., 1.]]]])
tensor([[[0.3133, 0.3133, 0.3133],
         [0.3133, 0.3133, 1.3133],
         [0.3133, 0.3133, 1.3133]]])
tensor(0.5355)
